In [1]:
## careercon kaggle

In [1]:
# initialization

%reset -f

import sys

import numpy as np, pandas as pd
import sklearn.metrics as skm
from sklearn.model_selection import train_test_split

# ignore warnings (only if you are the kind that would code when the world is burning)
import warnings
warnings.filterwarnings('ignore')

# some options
MAX_EVALS=5
randomseed = 1 # the value for the random state used at various points in the pipeline
pd.options.display.max_rows = 10 # specify if you want the full output in cells rather the truncated list
pd.options.display.max_columns = 200

# to display multiple outputs in a cell without usin print/display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# display wd files
import os as os
print('folder files: ', os.listdir(), '\n')
print('envir variables: ')
%who

from sklearn.model_selection import train_test_split
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
#from sklearn.ensemble import GradientBoostingClassifier

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

folder files:  ['.ipynb_checkpoints', 'car.pkl', 'careercon.ipynb', 'carhyperas.py', 'model.h5', 'pred.csv', 'preddf.csv', 'sample_submission.csv', 'sub.csv', 'X_test.csv', 'X_train.csv', 'y_train.csv', '__pycache__'] 

envir variables: 
InteractiveShell	 MAX_EVALS	 np	 os	 pd	 randomseed	 skm	 sys	 train_test_split	 
warnings	 


In [2]:
# global function to flatten columns after a grouped operation and aggregation
# outside all classes since it is added as an attribute to pandas DataFrames

def __my_flatten_cols(self, how="_".join, reset_index=True):
    how = (lambda iter: list(iter)[-1]) if how == "last" else how
    self.columns = [how(filter(None, map(str, levels))) for levels in self.columns.values] \
    if isinstance(self.columns, pd.MultiIndex) else self.columns
    return self.reset_index(drop=True) if reset_index else self
pd.DataFrame.my_flatten_cols = __my_flatten_cols

In [3]:
# input

train = pd.read_csv('X_train.csv')
test = pd.read_csv('X_test.csv')
trainres = pd.read_csv('y_train.csv')

In [4]:
trainres_enc = trainres.surface
enc = LabelEncoder()
enc.fit(trainres_enc)
trainres_enc = enc.transform(trainres_enc)

LabelEncoder()

In [7]:
train.head(2)

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,0_0,0,0,-0.75853,-0.63435,-0.10488,-0.10597,0.107650,0.017561,0.000767,-0.74857,2.1030,-9.7532
1,0_1,0,1,-0.75853,-0.63434,-0.10490,-0.10600,0.067851,0.029939,0.003385,0.33995,1.5064,-9.4128


In [8]:
np.array(np.unique(trainres_enc, return_counts=True)).T

array([[  0, 189],
       [  1, 779],
       [  2, 363],
       [  3,  21],
       [  4, 308],
       [  5, 732],
       [  6, 297],
       [  7, 514],
       [  8, 607]], dtype=int64)

In [9]:
## FOR AGGREGATED APPROACH

cols=['orientation_W', 'orientation_X', 'orientation_Y', 'orientation_Z', 'angular_velocity_X', 'angular_velocity_Y', 
      'angular_velocity_Z', 'linear_acceleration_X', 'linear_acceleration_Y', 'linear_acceleration_Z']

trainagg = (train.
            groupby(['series_id'])[cols].
            agg(['sum', 'mean', 'max', 'min', 'median', 'skew']).
            my_flatten_cols())
testagg = (test.
            groupby(['series_id'])[cols].
            agg(['sum', 'mean', 'max', 'min', 'median', 'skew']).
            my_flatten_cols())

In [10]:
trainagg['series_id'] = trainagg.index
testagg['series_id'] = testagg.index

In [11]:
train = train.merge(trainagg, how='left')
test = test.merge(testagg, how='left')

In [12]:
train.shape, test.shape

((487680, 73), (488448, 73))

In [13]:
# # remove highly correlated features to reduce further computation time
# print('correlation analysis is happening ...', '\n')
# # Create correlation matrix
# corr_matrix = train.corr().abs()
# # Select upper triangle of correlation matrix
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# # Find index of feature columns with correlation greater than 0.98
# to_drop = [column for column in upper.columns if any(upper[column] > 0.98)]        
# # Drop features
# #print(to_drop, '\n')
# train.drop(to_drop, axis=1, inplace=True)
# test.drop(to_drop, axis=1, inplace=True)
# print('correlation analysis completed ...', '\n')

In [14]:
trainnew = pd.merge(train, trainres, how='left')
trainnew['surface_enc'] = enc.transform(trainnew['surface'])
trainnew.drop(columns=['row_id', 'measurement_number', 'group_id', 'surface'], inplace=True)

In [93]:
test

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z,orientation_W_sum,orientation_W_mean,orientation_W_max,orientation_W_min,orientation_W_median,orientation_W_skew,orientation_X_sum,orientation_X_mean,orientation_X_max,orientation_X_min,orientation_X_median,orientation_X_skew,orientation_Y_sum,orientation_Y_mean,orientation_Y_max,orientation_Y_min,orientation_Y_median,orientation_Y_skew,orientation_Z_sum,orientation_Z_mean,orientation_Z_max,orientation_Z_min,orientation_Z_median,orientation_Z_skew,angular_velocity_X_sum,angular_velocity_X_mean,angular_velocity_X_max,angular_velocity_X_min,angular_velocity_X_median,angular_velocity_X_skew,angular_velocity_Y_sum,angular_velocity_Y_mean,angular_velocity_Y_max,angular_velocity_Y_min,angular_velocity_Y_median,angular_velocity_Y_skew,angular_velocity_Z_sum,angular_velocity_Z_mean,angular_velocity_Z_max,angular_velocity_Z_min,angular_velocity_Z_median,angular_velocity_Z_skew,linear_acceleration_X_sum,linear_acceleration_X_mean,linear_acceleration_X_max,linear_acceleration_X_min,linear_acceleration_X_median,linear_acceleration_X_skew,linear_acceleration_Y_sum,linear_acceleration_Y_mean,linear_acceleration_Y_max,linear_acceleration_Y_min,linear_acceleration_Y_median,linear_acceleration_Y_skew,linear_acceleration_Z_sum,linear_acceleration_Z_mean,linear_acceleration_Z_max,linear_acceleration_Z_min,linear_acceleration_Z_median,linear_acceleration_Z_skew
0,0_0,0,0,-0.025773,-0.98864,-0.148010,0.003350,-0.006524,-0.001071,-0.027390,0.10043,4.2061,-5.5439,0.40286,0.003147,0.003609,0.002654,0.003134,-0.142993,-3.303669,-0.025810,-0.025156,-0.026418,-0.025748,-0.389316,-126.54647,-0.988644,-0.98854,-0.98873,-0.988645,0.203751,-18.944760,-0.148006,-0.147480,-0.148720,-0.148010,-0.165049,0.127245,0.000994,0.231270,-0.225610,0.000418,0.123732,0.336571,0.002629,0.11486,-0.13797,0.002223,-0.203662,-0.380337,-0.002971,0.097635,-0.10259,-0.008017,0.095072,18.117709,0.141545,3.3996,-3.0939,0.150615,-0.185976,368.22865,2.876786,6.3266,0.20204,2.83825,0.125563,-1193.6338,-9.325264,-3.9960,-16.362,-9.3756,-0.047728
1,0_1,0,1,-0.025683,-0.98862,-0.148160,0.003439,-0.113960,0.083987,-0.060590,-0.70889,3.9905,-8.0273,0.40286,0.003147,0.003609,0.002654,0.003134,-0.142993,-3.303669,-0.025810,-0.025156,-0.026418,-0.025748,-0.389316,-126.54647,-0.988644,-0.98854,-0.98873,-0.988645,0.203751,-18.944760,-0.148006,-0.147480,-0.148720,-0.148010,-0.165049,0.127245,0.000994,0.231270,-0.225610,0.000418,0.123732,0.336571,0.002629,0.11486,-0.13797,0.002223,-0.203662,-0.380337,-0.002971,0.097635,-0.10259,-0.008017,0.095072,18.117709,0.141545,3.3996,-3.0939,0.150615,-0.185976,368.22865,2.876786,6.3266,0.20204,2.83825,0.125563,-1193.6338,-9.325264,-3.9960,-16.362,-9.3756,-0.047728
2,0_2,0,2,-0.025617,-0.98861,-0.148260,0.003571,-0.080518,0.114860,-0.037177,1.45710,2.2828,-11.2990,0.40286,0.003147,0.003609,0.002654,0.003134,-0.142993,-3.303669,-0.025810,-0.025156,-0.026418,-0.025748,-0.389316,-126.54647,-0.988644,-0.98854,-0.98873,-0.988645,0.203751,-18.944760,-0.148006,-0.147480,-0.148720,-0.148010,-0.165049,0.127245,0.000994,0.231270,-0.225610,0.000418,0.123732,0.336571,0.002629,0.11486,-0.13797,0.002223,-0.203662,-0.380337,-0.002971,0.097635,-0.10259,-0.008017,0.095072,18.117709,0.141545,3.3996,-3.0939,0.150615,-0.185976,368.22865,2.876786,6.3266,0.20204,2.83825,0.125563,-1193.6338,-9.325264,-3.9960,-16.362,-9.3756,-0.047728
3,0_3,0,3,-0.025566,-0.98862,-0.148170,0.003609,0.070067,0.033820,-0.035904,0.71096,1.8582,-12.2270,0.40286,0.003147,0.003609,0.002654,0.003134,-0.142993,-3.303669,-0.025810,-0.025156,-0.026418,-0.025748,-0.389316,-126.54647,-0.988644,-0.98854,-0.98873,-0.988645,0.203751,-18.944760,-0.148006,-0.147480,-0.148720,-0.148010,-0.165049,0.127245,0.000994,0.231270,-0.225610,0.000418,0.123732,0.336571,0.002629,0.11486,-0.13797,0.002223,-0.203662,-

In [94]:
test.drop(columns=['row_id', 'series_id', 'measurement_number'], inplace=True)

In [ ]:
YYY = test

In [15]:
XXX = trainnew.groupby(['series_id']).mean().reset_index()
XXY = XXX.surface_enc.values
XXX.drop(columns=['surface_enc', 'series_id'], inplace=True)

In [16]:
XXX.shape, XXY.shape

((3810, 70), (3810,))

In [17]:
XXX

,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z,orientation_W_sum,orientation_W_mean,orientation_W_max,orientation_W_min,orientation_W_median,orientation_W_skew,orientation_X_sum,orientation_X_mean,orientation_X_max,orientation_X_min,orientation_X_median,orientation_X_skew,orientation_Y_sum,orientation_Y_mean,orientation_Y_max,orientation_Y_min,orientation_Y_median,orientation_Y_skew,orientation_Z_sum,orientation_Z_mean,orientation_Z_max,orientation_Z_min,orientation_Z_median,orientation_Z_skew,angular_velocity_X_sum,angular_velocity_X_mean,angular_velocity_X_max,angular_velocity_X_min,angular_velocity_X_median,angular_velocity_X_skew,angular_velocity_Y_sum,angular_velocity_Y_mean,angular_velocity_Y_max,angular_velocity_Y_min,angular_velocity_Y_median,angular_velocity_Y_skew,angular_velocity_Z_sum,angular_velocity_Z_mean,angular_velocity_Z_max,angular_velocity_Z_min,angular_velocity_Z_median,angular_velocity_Z_skew,linear_acceleration_X_sum,linear_acceleration_X_mean,linear_acceleration_X_max,linear_acceleration_X_min,linear_acceleration_X_median,linear_acceleration_X_skew,linear_acceleration_Y_sum,linear_acceleration_Y_mean,linear_acceleration_Y_max,linear_acceleration_Y_min,linear_acceleration_Y_median,linear_acceleration_Y_skew,linear_acceleration_Z_sum,linear_acceleration_Z_mean,linear_acceleration_Z_max,linear_acceleration_Z_min,linear_acceleration_Z_median,linear_acceleration_Z_skew
0,-0.758666,-0.634008,-0.105474,-0.106470,-0.002481,-0.003306,0.007532,0.263418,2.984195,-9.320391,-13.628110,-0.106470,-0.105590,-0.107050,-0.106555,0.441564,-97.10922,-0.758666,-0.75822,-0.75953,-0.758530,-0.659082,-81.15298,-0.634008,-0.63306,-0.63456,-0.634270,0.603197,-13.500690,-0.105474,-0.104610,-0.106140,-0.105500,0.193309,-0.317527,-0.002481,0.107650,-0.160410,-0.005082,-0.342643,-0.423172,-0.003306,0.072698,-0.079404,-0.004037,0.016396,0.964051,0.007532,0.051720,-0.030181,0.006842,0.126373,33.717542,0.263418,2.85380,-1.864400,0.231665,0.132684,381.976947,2.984195,5.3864,0.075417,3.40755,-0.364964,-1193.010000,-9.320391,-6.26810,-12.5120,-9.42995,0.067391
1,-0.958606,0.241867,0.031650,-0.146876,0.004605,-0.007757,0.006206,0.121867,2.768193,-9.388899,-18.800070,-0.146876,-0.145870,-0.148090,-0.146910,-0.169549,-122.70162,-0.958606,-0.95837,-0.95896,-0.958595,-0.397289,30.95897,0.241867,0.24270,0.24074,0.241890,-0.422565,4.051239,0.031650,0.032341,0.030504,0.031689,-0.517180,0.589410,0.004605,0.283420,-0.254800,0.010344,-0.136062,-0.992868,-0.007757,0.112080,-0.134330,-0.006330,-0.246493,0.794404,0.006206,0.129150,-0.121610,0.003538,0.004702,15.599035,0.121867,5.10020,-3.193400,0.003571,0.759101,354.328697,2.768193,6.6850,-2.149200,2.75010,-0.183139,-1201.779100,-9.388899,-2.74490,-16.9280,-9.41380,-0.126848
2,-0.512057,-0.846171,-0.129371,-0.071082,0.002646,-0.009232,0.027989,0.149711,2.886745,-9.395783,-9.098478,-0.071082,-0.070378,-0.071535,-0.071139,0.511039,-65.54334,-0.512057,-0.50944,-0.51434,-0.512035,0.151971,-108.30988,-0.846171,-0.84490,-0.84779,-0.846210,-0.161786,-16.559440,-0.129371,-0.128520,-0.130300,-0.129405,-0.034406,0.338645,0.002646,0.141920,-0.152710,-0.003120,0.205228,-1.181655,-0.009232,0.091946,-0.107810,-0.010879,-0.077528,3.582611,0.027989,0.088730,-0.015697,0.028323,0.069123,19.163063,0.149711,1.85330,-2.593000,0.174515,-0.480996,369.503305,2.886745,6.2105,-1.254000,3.03375,-0.266815,-1202.660200,-9.395783,-5.74420,-12.4990,-9.37440,0.085877
3,-0.939169,0.310140,0.038955,-0.142319,0.000624,-0.002804,0.007887,0.201791,2.657922,-9.451164,-18.216810,-0.142319,-0.139340,-0.144370,-0.142510,0.175628,-120.21364,-0.939169,-0.93884,-0.93968,-0.939170,-0.096106,39.69794,0.310140,0.31147,0.30943,0.310115,1.230981,4.986241,0.038955,0.039799,0.037922,0.038889,0.097680,0.079866,0.000624,0.519130,-0.401520,0.006709,-0.003575,-0.358926,-0.002804,0.135780,-0.168150,0.000518,-0.320948,1.009490,

In [18]:
dummy_train, dummy_test, dummy_ytrain, dummy_ytest = train_test_split(XXX, XXY,
                                                    stratify=XXY,
                                                    test_size=0.33, random_state=10)

In [21]:
np.array(np.unique(dummy_ytrain, return_counts=True)).T

array([[  0, 127],
       [  1, 522],
       [  2, 243],
       [  3,  14],
       [  4, 206],
       [  5, 490],
       [  6, 199],
       [  7, 344],
       [  8, 407]], dtype=int64)

In [22]:
np.array(np.unique(dummy_ytest, return_counts=True)).T

array([[  0,  62],
       [  1, 257],
       [  2, 120],
       [  3,   7],
       [  4, 102],
       [  5, 242],
       [  6,  98],
       [  7, 170],
       [  8, 200]], dtype=int64)

In [61]:
import xgboost as xgb

def get_xgb_imp(xgb):
    imp_vals = xgb.get_fscore()
    feats_imp = pd.DataFrame(imp_vals,index=np.arange(2)).T
    feats_imp.iloc[:,0]= feats_imp.index    
    feats_imp.columns=['feature','importance']
    feats_imp.sort_values('importance',inplace=True,ascending=False)
    feats_imp.reset_index(drop=True,inplace=True)
    return feats_imp

def quick_model(train, valid, ytrain, yvalid):
    weights_dict = {0:1.3, 1:1, 2:1.2, 3:2, 4:1.3, 5:1, 6:1.5, 7:1.2, 8:1}
    ytrain_weights = np.copy(ytrain)
    yvalid_weights = np.copy(yvalid)
    for k, v in weights_dict.items(): 
        ytrain_weights[ytrain==k]=v
        yvalid_weights[yvalid==k]=v
    
    feat_names = train.columns.values

    xg_train = xgb.DMatrix(train, label=ytrain, weight=ytrain_weights)
    xg_test = xgb.DMatrix(valid, label=yvalid, weight=yvalid_weights)

    # setup parameters for xgboost
    param = {'objective':'multi:softprob', 'max_depth':15, 'silent':1, 'nthread':-1, 'num_class':9, 'subsample':0.85, 
             'colsample_bytree':0.7, 'learning_rate':0.05, 'eval_metric':['mlogloss', 'merror'], 'n_jobs': -1,
            'colsample_bylevel':0.6, 'tree_method':'hist', 'seed':10, 'grow_policy':'lossguide', 'max_delta_step':1,
            'max_bin':512}

    watchlist = [(xg_train, 'train'), (xg_test, 'test')]
    num_round = 100
    model = xgb.train(param, xg_train, num_round, watchlist, early_stopping_rounds=20, verbose_eval=10)
    # get prediction
    pred_probs = model.predict(xg_test)
    pred = np.argmax(pred_probs, 1)
    error_rate = np.sum(pred != yvalid) / yvalid.shape[0]
    print('Test error using softprob = {}'.format(error_rate))

    print(skm.accuracy_score(y_pred=pred, y_true=yvalid))
    print(skm.confusion_matrix(y_pred=pred, y_true=yvalid))
    return model, feat_names, xg_train, xg_test, pred, pred_probs

In [62]:
model, feat_names, xg_train, xg_test, pred, pred_probs = quick_model(train=dummy_train.copy(),
                                                                    valid=dummy_test.copy(),
                                                                    ytrain=dummy_ytrain.copy(),
                                                                    yvalid=dummy_ytest.copy())

[0]	train-mlogloss:2.1375	train-merror:0.226422	test-mlogloss:2.14645	test-merror:0.327273
Multiple eval metrics have been passed: 'test-merror' will be used for early stopping.

Will train until test-merror hasn't improved in 20 rounds.
[10]	train-mlogloss:1.57506	train-merror:0.069369	test-mlogloss:1.68153	test-merror:0.21502
[20]	train-mlogloss:1.0937	train-merror:0.023383	test-mlogloss:1.29887	test-merror:0.186561
[30]	train-mlogloss:0.722813	train-merror:0.007794	test-mlogloss:1.00785	test-merror:0.167589
[40]	train-mlogloss:0.484351	train-merror:0.003897	test-mlogloss:0.810926	test-merror:0.156522
[50]	train-mlogloss:0.335085	train-merror:0.000779	test-mlogloss:0.685116	test-merror:0.149407
[60]	train-mlogloss:0.237882	train-merror:0.00039	test-mlogloss:0.599325	test-merror:0.150198
[70]	train-mlogloss:0.173756	train-merror:0	test-mlogloss:0.540165	test-merror:0.148617
[80]	train-mlogloss:0.130937	train-merror:0	test-mlogloss:0.499974	test-merror:0.144664
[90]	train-mlogloss:0.10

In [59]:
from hyperopt import hp, tpe, STATUS_OK, fmin, Trials
from hyperopt.fmin import fmin
from hyperopt.pyll.stochastic import sample

class xgbclass():
    """ mode = c['C', 'R'] (C==classification, R==regression)
    self.cust_score
    """
    
    ################################################### INIT ###############################################
    def __init__(self, train, valid, ytrain, yvalid, mode=['C']):
        self.mode=mode
        self.randomseed = 1
        self.N_FOLDS = 3
        self.trials = Trials()
        self.MAX_EVALS = 10
        self.yvalid = yvalid
        
        feat_names = train.columns.values

        weights_dict = {0:1.3, 1:1, 2:1.2, 3:2, 4:1.3, 5:1, 6:1.5, 7:1.2, 8:1}
        
        ytrain_weights = np.copy(ytrain)
        yvalid_weights = np.copy(yvalid)
        for k, v in weights_dict.items(): 
            ytrain_weights[ytrain==k]=v
            yvalid_weights[yvalid==k]=v
            
        self.dtrain = xgb.DMatrix(data=train, feature_names=train.columns.values, 
                                  label=ytrain, weight=ytrain_weights)
        self.dtest = xgb.DMatrix(data=valid, feature_names=valid.columns.values, 
                                 label=yvalid, weight=yvalid_weights)

        self.optimize()
    ####################################################################################################
    
    def cust_score(self, preds, dtrain):
        labels = dtrain.get_label()
        score = skm.log_loss(y_pred=preds, y_true=labels)
        return 'score', score
    
    # function to be minimized and sent to the optimize function of hyperopt
    def xgb_score(self, params):
        # Make sure parameters that need to be integers are integers
        for parameter_name in ['max_depth', 'max_bin']:
            params[parameter_name] = int(params[parameter_name])
            
        ## to tune on cv results (the right method)
        xgb_cv = xgb.cv(params=params, num_boost_round=50, nfold=self.N_FOLDS, dtrain=self.dtrain, early_stopping_rounds=10,
                       maximize=False, stratified=True, verbose_eval=25, metrics=['merror', 'mlogloss'])#, feval=self.cust_score 
        num_rounds = len(xgb_cv['test-merror-mean'])
        bst_score = xgb_cv['test-merror-mean'][num_rounds-1]
        print('evaluation metric score of iteration is: ', bst_score, '\n')
        print('validation accuracy is: ', 1-xgb_cv['test-merror-mean'][num_rounds-1])
        return {'loss': bst_score, 'status': STATUS_OK, 'params': params, 'num_boost': num_rounds-1, 
                'valid_acc': 1-xgb_cv['test-merror-mean'][num_rounds-1]}        
    
    # function to do hyperparameter tuning with hyperopt (bayesian based method)
    def optimize(self):
        # space to be traversed for the hyperopt function
        space = {
            'base_score' : hp.uniform('base_score', 0.45, 0.55),
             #'max_depth' : hp.choice('max_depth', np.arange(3, 8, dtype=int)),
            'max_depth' : hp.quniform('max_depth', 4, 15, 1),
             'min_child_weight' : hp.quniform('min_child_weight', 2, 10, 0.1),
             'subsample' : hp.uniform('subsample', 0.7, 0.95),
            'alpha': hp.uniform('alpha', 0, 0.1),
             'gamma' : hp.uniform('gamma', 0, 0.1),
            'lambda' : hp.uniform ('lambda', 0, 0.1),
             'colsample_bytree' : hp.uniform('colsample_bytree', 0.7, 0.85),
            'colsample_bylevel': hp.uniform('colsample_bylevel', 0.6, 0.85),
            'objective' : 'multi:softprob',
            'num_class':9,
            'grow_policy': 'lossguide', #hp.choice('grow_policy', ['depthwise', 'lossguide']),
            'max_bin': hp.quniform('max_bin', 250, 500, 25),
            'max_delta_step': hp.quniform('max_delta_step', 0, 5, 1),
            'booster': 'gbtree', #hp.choice('booster', ['gbtree', 'dart']),
            'tree_method': 'exact', #hp.choice('tree_method', ['exact', '']),
            'n_jobs': -1,
            'learning_rate':0.1
        }
        self.best = fmin(self.xgb_score, space, algo=tpe.suggest, trials=self.trials, max_evals=self.MAX_EVALS,
                    rstate=np.random.RandomState(self.randomseed))
        self.num_rounds = self.trials.best_trial['result']['num_boost']
        return None # results of all the iterations, the best one and the number of rounds for the best run
    
    # train and return a model with the best params
    def xgb_train_and_predict(self):
        self.model = xgb.train(self.trials.best_trial['result']['params'], dtrain=self.dtrain, maximize=False, 
                               num_boost_round=self.num_rounds) #, feval=self.cust_score)
        self.predprobs = self.model.predict(self.dtest)
        self.pred = np.asarray([np.argmax(line) for line in self.predprobs])
        print(skm.accuracy_score(y_pred=self.pred, y_true=self.yvalid), '\n')
        print(skm.confusion_matrix(y_true=self.yvalid, y_pred=xgbmod.pred), '\n')
        return None
    
    # function to return cv results for train dataset (recall/precision/f1/accuracy)
    def xgb_cv(self):
        best=self.trials.best_trial['result']['params']
        model = xgb.XGBClassifier(**best, silent=True)
        xgb_cv_scores = sklearn.model_selection.cross_val_predict(model, self.dtrain, self.trainres, cv=5)
        print('recall: ', sklearn.metrics.recall_score(y_pred=xgb_cv_scores, y_true=self.trainres))
        print('precision: ', sklearn.metrics.precision_score(y_pred=xgb_cv_scores, y_true=self.trainres))
        print('f1: ', sklearn.metrics.f1_score(y_pred=xgb_cv_scores, y_true=self.trainres))
        print('accuracy: ', sklearn.metrics.accuracy_score(y_pred=xgb_cv_scores, y_true=self.trainres))
        return None
    
    def get_xgb_imp(self):
        imp_vals = self.model.get_fscore()
        feats_imp = pd.DataFrame(imp_vals,index=np.arange(2)).T
        feats_imp.iloc[:,0]= feats_imp.index
        feats_imp.columns=['feature','importance']
        feats_imp.sort_values('importance',inplace=True,ascending=False)
        feats_imp.reset_index(drop=True,inplace=True)
        return feats_imp

In [60]:
xgbmod = xgbclass(train=dummy_train.copy(), valid=dummy_test.copy(), ytrain=dummy_ytrain.copy(), yvalid=dummy_ytest.copy())

[0]	train-merror:0.330054+0.0182989	train-mlogloss:2.05138+0.00389012	test-merror:0.393254+0.0205168	test-mlogloss:2.06561+0.00416511

[25]	train-merror:0.082618+0.000906007	train-mlogloss:0.547831+0.01529	test-merror:0.229904+0.0187176	test-mlogloss:0.819828+0.0448178

[49]	train-merror:0.0290343+0.00121875	train-mlogloss:0.28327+0.00803519	test-merror:0.210058+0.016436	test-mlogloss:0.654663+0.0538614

evaluation metric score of iteration is: 
0.21005766666666667
validation accuracy is: 
0.7899423333333333
[0]	train-merror:0.289952+0.00535177	train-mlogloss:2.08427+0.000877262	test-merror:0.404585+0.0231592	test-mlogloss:2.10267+0.00197732

[25]	train-merror:0.0126677+0.00101644	train-mlogloss:0.407789+0.00490342	test-merror:0.19254+0.0136963	test-mlogloss:0.780082+0.0504018

[49]	train-merror:0.00117033+0.000829201	train-mlogloss:0.130321+0.00212302	test-merror:0.180439+0.0194919	test-mlogloss:0.570148+0.0659364

evaluation metric score of iteration is: 
0.18043866666666666
validati

In [63]:
xgbmod.xgb_train_and_predict()

xgbmod.best
model2=xgbmod.model

pd.DataFrame(np.array(np.unique(xgbmod.pred, return_counts=True)).T)

feature_importance_df1 = xgbmod.get_xgb_imp()
xgb.plot_importance(model, max_num_features = 20)

0.856120826709062 

[[ 47   6   0   0   2   0   0   2   5]
 [  4 215   3   0   4   5   4  13   9]
 [  1   4  97   0   0   6   0   3   9]
 [  0   0   0   3   0   1   1   0   2]
 [  0   5   0   0  91   2   0   1   3]
 [  3   8   3   0   2 212   5   3   6]
 [  0   3   0   0   0   3  91   1   0]
 [  2   3   2   0   0   2   2 157   2]
 [  2   5   1   0   0  17   0  11 164]] 



{'alpha': 0.07739865491608339,
 'base_score': 0.45148537679535017,
 'colsample_bylevel': 0.8219456271136347,
 'colsample_bytree': 0.770458686268976,
 'gamma': 0.0562367981858677,
 'lambda': 0.0816621449648167,
 'max_bin': 300.0,
 'max_delta_step': 2.0,
 'max_depth': 11.0,
 'min_child_weight': 2.8000000000000003,
 'subsample': 0.8624294820509246}

,0,1
0,0,59
1,1,249
2,2,106
3,3,3
4,4,99
5,5,248
6,6,103
7,7,191
8,8,200


In [92]:
params_new = {'alpha': 0.07739865491608339,
'base_score': 0.45148537679535017,
'colsample_bylevel': 0.8219456271136347,
'colsample_bytree': 0.770458686268976,
'gamma': 0.0562367981858677,
'lambda': 0.0816621449648167,
'max_bin': 300,
'max_delta_step': 2,
'max_depth': 30,
'min_child_weight': 2.8000000000000003,
'subsample': 0.8624294820509246,
'metrics':['mlogloss', 'merror'],
'objective' : 'multi:softprob',
'num_class':9,
'grow_policy': 'lossguide',
'learning_rate':0.01,
'n_jobs':-1}

watchlist = [(xgbmod.dtrain, 'train'), (xgbmod.dtest, 'test')]
numround=500
model3 = xgb.train(params_new, xgbmod.dtrain, numround, watchlist, maximize=False, 
                   early_stopping_rounds=100, verbose_eval=50)
dummypredprobs = model3.predict(xgbmod.dtest)
dummypred = np.asarray([np.argmax(line) for line in dummypredprobs])
print(skm.accuracy_score(y_pred=dummypred, y_true=dummy_ytest), '\n')
print(skm.confusion_matrix(y_true=dummy_ytest, y_pred=dummypred), '\n')

[0]	train-merror:0.199532	test-merror:0.303557
Multiple eval metrics have been passed: 'test-merror' will be used for early stopping.

Will train until test-merror hasn't improved in 100 rounds.
[50]	train-merror:0.046376	test-merror:0.167589
[100]	train-merror:0.026111	test-merror:0.158893
[150]	train-merror:0.01403	test-merror:0.149407
[200]	train-merror:0.008574	test-merror:0.147036
[250]	train-merror:0.006625	test-merror:0.148617
[300]	train-merror:0.003897	test-merror:0.145455
[350]	train-merror:0.002338	test-merror:0.143083
[400]	train-merror:0.001559	test-merror:0.143083
[450]	train-merror:0.000779	test-merror:0.13834
[499]	train-merror:0.00039	test-merror:0.137549
0.8640699523052464 

[[ 51   5   0   0   1   0   0   1   4]
 [  3 221   2   0   5   5   4  11   6]
 [  1   5  97   0   0   4   0   3  10]
 [  0   0   0   4   0   1   1   0   1]
 [  0   4   0   0  92   1   0   1   4]
 [  3   7   3   0   3 212   6   2   6]
 [  0   3   0   0   0   3  91   1   0]
 [  2   3   2   0   1   2

In [88]:
X = np.array(dummy_train)
XV = np.array(dummy_test)
Y = to_categorical(dummy_ytrain)
YV = to_categorical(dummy_ytest)

In [39]:
import pickle

car = open('./car.pkl','wb')
pickle.dump(X, car)
pickle.dump(Y, car)
pickle.dump(XV, car)
pickle.dump(YV, car)
car.close()

In [40]:
%run -i carhyperas.py

Using TensorFlow backend.


>>> Imports:
#coding=utf-8

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Flatten, LSTM, SpatialDropout1D
except:
    pass

try:
    from tensorflow.keras.models import Sequential
except:
    pass

try:
    from tensorflow.keras.callbacks import EarlyStopping
except:
    pass

try:
    import pickle
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'activation': hp.choice('activation', ['relu', 'sigmoid', 'tanh', 'elu']),
        'Dense': hp.choice('Dense', [50, 100, 250, 500, 1000, 2000]),
        'activation_1': hp.choice('activation_1', ['relu', 'sigmoid', 'tanh', 'elu']),
        'Dropout': hp.uniform('Dropout', 0, 0.7),
        'Dense_1': hp.choice('Dense_1', 

In [41]:
from tensorflow.keras.models import load_model
model = load_model('model.h5')
best_run

In [47]:
model.fit(X,Y,batch_size=50, epochs=100, verbose=1, shuffle=True, validation_data=(XV,YV))

Train on 2552 samples, validate on 1258 samples
Epoch 1/100
2552/2552 [==============================] - 0s 116us/sample - loss: 1.9432 - accuracy: 0.2308 - val_loss: 1.9279 - val_accuracy: 0.2393
Epoch 2/100
2552/2552 [==============================] - 0s 111us/sample - loss: 1.9295 - accuracy: 0.2241 - val_loss: 1.9278 - val_accuracy: 0.2409
Epoch 3/100
2552/2552 [==============================] - 0s 135us/sample - loss: 1.9268 - accuracy: 0.2328 - val_loss: 1.9327 - val_accuracy: 0.2345
Epoch 4/100
2552/2552 [==============================] - 0s 121us/sample - loss: 1.9336 - accuracy: 0.2300 - val_loss: 1.9261 - val_accuracy: 0.2480
Epoch 5/100
2552/2552 [==============================] - 0s 118us/sample - loss: 1.9291 - accuracy: 0.2339 - val_loss: 1.9273 - val_accuracy: 0.2432
Epoch 6/100
2552/2552 [==============================] - 0s 113us/sample - loss: 1.9256 - accuracy: 0.2390 - val_loss: 1.9192 - val_accuracy: 0.2504
Epoch 7/100
2552/2552 [==============================] - 0

In [48]:
model.evaluate(XV,YV)

1258/1258 [==============================] - 0s 60us/sample - loss: 1.7769 - accuracy: 0.3347


[1.7768893507213017, 0.33465818]

In [99]:
X = np.array(XXX)
Y = to_categorical(XXY)
testnew = test.drop(columns=['row_id', 'series_id', 'measurement_number'])
Z = np.array(testnew.values)

In [ ]:
model.fit(X,Y,batch_size=2, epochs=10, verbose=1, shuffle=True)

Epoch 1/10
3810/3810 [==============================] - 147s 39ms/sample - loss: 1.9133 - accuracy: 0.2793
Epoch 2/10
3810/3810 [==============================] - 150s 39ms/sample - loss: 1.9271 - accuracy: 0.2819
Epoch 3/10
3810/3810 [==============================] - 153s 40ms/sample - loss: 1.9153 - accuracy: 0.2740
Epoch 4/10
3810/3810 [==============================] - 150s 39ms/sample - loss: 1.9116 - accuracy: 0.2858
Epoch 5/10
3810/3810 [==============================] - 151s 40ms/sample - loss: 1.9006 - accuracy: 0.2895
Epoch 6/10
3810/3810 [==============================] - 151s 40ms/sample - loss: 1.9095 - accuracy: 0.2738
Epoch 7/10
3810/3810 [==============================] - 150s 39ms/sample - loss: 1.9114 - accuracy: 0.2934
Epoch 8/10
1428/3810 [==========>...................] - ETA: 1:33 - loss: 1.9166 - accuracy: 0.2745

In [101]:
pred=model.predict_classes(Z)

In [84]:
preddf = pd.DataFrame({'surface':pred})
preddf=pd.concat([test[['series_id']].reset_index(drop=True), preddf], ignore_index=True, axis=1)
preddf_agg = preddf.groupby([0], as_index=False).agg(lambda x:x.value_counts().index[0]).my_flatten_cols()
preddf_agg.columns = ['series_id', 'surface']
preddf_agg['surface'] = enc.inverse_transform(preddf_agg['surface'])
preddf_agg.to_csv('sub.csv', index=False)

,0,1
0,0,5
1,0,5
2,0,5
3,0,5
4,0,5
...,...,...
488443,3815,8
488444,3815,8
488445,3815,8
488446,3815,8
